# SAP Report Cleaner

Dieses Notebook bereinigt SAP-Reports, die als pseudo-XLS-Dateien (eigentlich tab-getrennte Texte) vorliegen.

## Funktionen:
- Entfernt Datumsinformationen aus Zeile 1
- Filtert nur Spalten C bis Q
- Entfernt Summenzeilen (markiert mit "*" in Spalte B)
- Entfernt leere Zeilen
- Entfernt Zeilen ohne Materialnummer in Spalte C
- Bereinigt Zahlenformate für pandas
- Konvertiert Datumsformate (DD.MM.YY)
- Exportiert als saubere CSV-Datei
- Speichert gelöschte Zeilen zur Nachverfolgung


In [ ]:
# Installation und Imports
import pandas as pd
import numpy as np
from datetime import datetime
import re
import os
from google.colab import files
import io


In [ ]:
# openpyxl für Excel-Export installieren (falls nicht vorhanden)
try:
    import openpyxl
except ImportError:
    print("Installiere openpyxl für Excel-Export...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl", "-q"])
    import openpyxl


## 1. Datei hochladen


In [ ]:
# Datei hochladen
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"Hochgeladene Datei: {file_name}")


## 2. Datei einlesen und vorbereiten


In [ ]:
# Datei als tab-getrennten Text einlesen
with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

print(f"Gesamtanzahl Zeilen: {len(lines)}")
print(f"Erste 5 Zeilen:")
for i, line in enumerate(lines[:5], 1):
    print(f"{i}: {repr(line[:100])}")


## 3. Daten bereinigen


In [ ]:
# Zeilen in Spalten aufteilen (tab-getrennt)
data_rows = []
deleted_rows = []  # Für Nachverfolgung gelöschter Zeilen

for idx, line in enumerate(lines, start=1):
    # Tab-getrennte Werte extrahieren
    row = line.rstrip('\n\r').split('\t')
    
    # Zeile 1 überspringen (Datum)
    if idx == 1:
        deleted_rows.append({
            'Zeile': idx,
            'Grund': 'Datum in Zeile 1',
            'Inhalt': line[:100]
        })
        continue
    
    # Leere Zeilen überspringen
    if not line.strip():
        deleted_rows.append({
            'Zeile': idx,
            'Grund': 'Leere Zeile',
            'Inhalt': ''
        })
        continue
    
    # Mindestens 17 Spalten benötigt (A=0 bis Q=16)
    if len(row) < 17:
        # Prüfen ob es eine komplett leere Zeile ist
        if all(not cell.strip() for cell in row):
            deleted_rows.append({
                'Zeile': idx,
                'Grund': 'Komplett leere Zeile',
                'Inhalt': line[:100]
            })
            continue
        else:
            # Zeile mit weniger Spalten - mit leeren Werten auffüllen
            row.extend([''] * (17 - len(row)))
    
    data_rows.append({
        'original_index': idx,
        'row': row
    })

print(f"Verarbeitete Zeilen: {len(data_rows)}")
print(f"Gelöschte Zeilen: {len(deleted_rows)}")


In [ ]:
# Spaltenüberschriften finden (sollten in Zeile 4 stehen)
# Erwartete Spalten: Material, Functional Loc., Equipment, Material Description, Work Ctr, 
# Withdrawn, W/o resrv., Reserved, Reserv.ref, Pstng Date, Order, ID, Message, ICt, Customer

expected_headers = [
    'Material', 'Functional Loc.', 'Equipment', 'Material Description', 'Work Ctr',
    'Withdrawn', 'W/o resrv.', 'Reserved', 'Reserv.ref', 'Pstng Date',
    'Order', 'ID', 'Message', 'ICt', 'Customer'
]

# Header-Zeile finden (normalerweise Zeile 4, Index 3 in 0-basiert)
header_row_idx = None
for i, data_row in enumerate(data_rows):
    row = data_row['row']
    # Prüfen ob diese Zeile die Header enthält
    if len(row) >= 17:
        # Spalten C bis Q (Index 2-16) extrahieren
        potential_headers = [row[j].strip() if j < len(row) else '' for j in range(2, 17)]
        # Prüfen ob "Material" in Spalte C steht
        if potential_headers[0] == 'Material' or 'Material' in potential_headers[0]:
            header_row_idx = i
            actual_headers = potential_headers
            break

if header_row_idx is None:
    # Fallback: Verwende erwartete Header
    print("Warnung: Header-Zeile nicht gefunden, verwende erwartete Header")
    actual_headers = expected_headers
    header_row_idx = 3  # Standard: Zeile 4
else:
    print(f"Header-Zeile gefunden in Zeile {data_rows[header_row_idx]['original_index']}")
    print(f"Gefundene Header: {actual_headers}")

# Header-Zeile aus data_rows entfernen
if header_row_idx is not None:
    header_data = data_rows.pop(header_row_idx)
    deleted_rows.append({
        'Zeile': header_data['original_index'],
        'Grund': 'Header-Zeile',
        'Inhalt': '\t'.join(header_data['row'])[:100]
    })


In [ ]:
# Daten filtern und bereinigen
cleaned_data = []
filtered_out = []

for data_row in data_rows:
    original_idx = data_row['original_index']
    row = data_row['row']
    
    # Spalten C bis Q extrahieren (Index 2-16)
    if len(row) < 17:
        row.extend([''] * (17 - len(row)))
    
    cols_c_to_q = [row[i].strip() if i < len(row) else '' for i in range(2, 17)]
    
    # Prüfung 1: Spalte B (Index 1) hat "*" UND Spalte D (Index 3) ist leer -> Summenzeile
    col_b = row[1].strip() if len(row) > 1 else ''
    col_d = row[3].strip() if len(row) > 3 else ''
    
    if col_b == '*' and not col_d:
        filtered_out.append({
            'Zeile': original_idx,
            'Grund': 'Summenzeile (* in Spalte B, leer in Spalte D)',
            'Inhalt': '\t'.join(row)[:100]
        })
        continue
    
    # Prüfung 2: Keine Materialnummer in Spalte C (Index 2, erste Spalte in cols_c_to_q)
    material_num = cols_c_to_q[0] if len(cols_c_to_q) > 0 else ''
    if not material_num or material_num == '':
        filtered_out.append({
            'Zeile': original_idx,
            'Grund': 'Keine Materialnummer in Spalte C',
            'Inhalt': '\t'.join(row)[:100]
        })
        continue
    
    # Prüfung 3: Komplett leere Zeile in Spalten C-Q
    if all(not cell.strip() for cell in cols_c_to_q):
        filtered_out.append({
            'Zeile': original_idx,
            'Grund': 'Komplett leere Zeile in Spalten C-Q',
            'Inhalt': '\t'.join(row)[:100]
        })
        continue
    
    # Zeile behalten
    cleaned_data.append(cols_c_to_q)

print(f"Bereinigte Datenzeilen: {len(cleaned_data)}")
print(f"Gefilterte Zeilen: {len(filtered_out)}")

# Gelöschte Zeilen zur Nachverfolgung hinzufügen
deleted_rows.extend(filtered_out)


In [ ]:
# DataFrame erstellen
df = pd.DataFrame(cleaned_data, columns=expected_headers)

print(f"DataFrame Shape: {df.shape}")
print(f"\nErste 5 Zeilen:")
print(df.head())
print(f"\nSpalten: {df.columns.tolist()}")


## 4. Datenformate bereinigen


In [ ]:
# Funktion zum Bereinigen von Zahlen (entfernt Leerzeichen, Tausenderpunkte, etc.)
def clean_number(value):
    if pd.isna(value) or value == '' or str(value).strip() == '':
        return None
    value_str = str(value).strip()
    
    # Alle Leerzeichen entfernen
    value_str = value_str.replace(' ', '')
    
    # Negative Zahlen erkennen (z.B. "-10")
    is_negative = value_str.startswith('-')
    if is_negative:
        value_str = value_str[1:]
    
    # Tausenderpunkte entfernen (z.B. "3.500" -> "3500")
    # SAP verwendet oft Punkte als Tausender-Trennzeichen
    if '.' in value_str:
        # Prüfen ob es ein Tausenderpunkt ist
        # Pattern: Zahl mit Punkt gefolgt von genau 3 Ziffern am Ende = Tausenderpunkt
        # z.B. "3.500" oder "12.345"
        parts = value_str.rsplit('.', 1)  # Split vom Ende
        if len(parts) == 2 and len(parts[1]) == 3 and parts[1].isdigit():
            # Tausenderpunkt entfernen
            value_str = parts[0] + parts[1]
        elif ',' in value_str:
            # Falls Komma vorhanden, könnte es ein Dezimaltrennzeichen sein
            # Aber laut Anforderung sind es Integer, also ignorieren wir Dezimalstellen
            value_str = value_str.replace(',', '')
            value_str = value_str.replace('.', '')
        else:
            # Punkt vorhanden, aber nicht als Tausenderpunkt erkannt -> entfernen
            value_str = value_str.replace('.', '')
    
    # Komma entfernen (falls noch vorhanden)
    value_str = value_str.replace(',', '')
    
    try:
        # Als Integer konvertieren (laut Anforderung sind es Integer)
        result = int(value_str) if value_str else None
        return -result if is_negative and result is not None else result
    except (ValueError, TypeError):
        return None

# Funktion zum Konvertieren von Datum (DD.MM.YY oder DD.MM.YYYY)
def convert_date(value):
    if pd.isna(value) or value == '' or str(value).strip() == '':
        return None
    value_str = str(value).strip()
    try:
        # Format: DD.MM.YY (8 Zeichen)
        if len(value_str) == 8 and value_str.count('.') == 2:
            day, month, year = value_str.split('.')
            # Jahr 00-99 -> 2000-2099
            year_int = int(year)
            if year_int < 50:
                year_int += 2000
            else:
                year_int += 1900
            return f"{day}.{month}.{year_int:04d}"
        # Format: DD.MM.YYYY (10 Zeichen, bereits vollständig)
        elif len(value_str) == 10 and value_str.count('.') == 2:
            return value_str
    except:
        pass
    return value_str  # Falls Konvertierung fehlschlägt, Original zurückgeben


In [ ]:
# Spaltenzuordnung:
# Material (C) bleibt als Text (Identifier)
# D, F, G, L, Q sind Text: Functional Loc., Material Description, Work Ctr, Pstng Date, Customer
# L ist Datum (Index 10) - "Pstng Date" 
# Rest sind Zahlen (Integer)

# Spalten die Text bleiben sollen (inkl. Material als Identifier)
text_columns = ['Material', 'Functional Loc.', 'Equipment', 'Material Description', 'Work Ctr', 'Customer']
# Spalte die Datum ist
date_column = 'Pstng Date'
# Rest sind Zahlen
numeric_columns = [col for col in df.columns if col not in text_columns and col != date_column]

print(f"Text-Spalten: {text_columns}")
print(f"Datum-Spalte: {date_column}")
print(f"Zahlen-Spalten: {numeric_columns}")


In [ ]:
# Datum-Spalte konvertieren
if date_column in df.columns:
    df[date_column] = df[date_column].apply(convert_date)
    print(f"Datum-Spalte '{date_column}' konvertiert")
    print(f"Beispielwerte: {df[date_column].head().tolist()}")

# Zahlen-Spalten bereinigen
for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_number)
        print(f"Zahlen-Spalte '{col}' bereinigt")

print("\nDataFrame Info:")
print(df.info())
print("\nErste 5 Zeilen nach Bereinigung:")
print(df.head())


## 5. Gelöschte Zeilen zur Nachverfolgung


In [ ]:
# DataFrame für gelöschte Zeilen erstellen
df_deleted = pd.DataFrame(deleted_rows)

print(f"Anzahl gelöschter Zeilen: {len(df_deleted)}")
print(f"\nGelöschte Zeilen nach Grund:")
print(df_deleted['Grund'].value_counts())
print(f"\nErste 10 gelöschte Zeilen:")
print(df_deleted.head(10))


## 6. Export als CSV


In [ ]:
# Output-Dateinamen erstellen
base_name = file_name.rsplit('.', 1)[0] if '.' in file_name else file_name
output_csv = f"{base_name}_cleaned.csv"
output_excel = f"{base_name}_cleaned.xlsx"
output_deleted_csv = f"{base_name}_deleted_rows.csv"

# Bereinigte Daten als CSV exportieren
df.to_csv(output_csv, index=False, encoding='utf-8-sig', sep=';')
print(f"Bereinigte Daten als CSV gespeichert: {output_csv}")
print(f"Anzahl Zeilen: {len(df)}")
print(f"Anzahl Spalten: {len(df.columns)}")

# Excel-Datei mit mehreren Sheets erstellen (bereinigte Daten + gelöschte Zeilen)
try:
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Bereinigte Daten', index=False)
        if len(df_deleted) > 0:
            df_deleted.to_excel(writer, sheet_name='Gelöschte Zeilen', index=False)
    print(f"\nExcel-Datei mit mehreren Sheets erstellt: {output_excel}")
    print(f"  - Sheet 'Bereinigte Daten': {len(df)} Zeilen")
    if len(df_deleted) > 0:
        print(f"  - Sheet 'Gelöschte Zeilen': {len(df_deleted)} Zeilen")
except ImportError:
    print("\nWarnung: openpyxl nicht installiert, Excel-Export übersprungen")
    print("Installieren mit: !pip install openpyxl")
    # Fallback: Gelöschte Zeilen als separate CSV
    if len(df_deleted) > 0:
        df_deleted.to_csv(output_deleted_csv, index=False, encoding='utf-8-sig', sep=';')
        print(f"Gelöschte Zeilen als CSV gespeichert: {output_deleted_csv}")
        print(f"Anzahl gelöschter Zeilen: {len(df_deleted)}")


In [ ]:
# Dateien zum Download bereitstellen
files.download(output_csv)
try:
    files.download(output_excel)
    print(f"📊 Excel-Datei: {output_excel}")
except:
    pass

if len(df_deleted) > 0 and not os.path.exists(output_excel):
    files.download(output_deleted_csv)
    print(f"🗑️  Gelöschte Zeilen: {output_deleted_csv}")

print("\n✅ Export abgeschlossen!")
print(f"📊 Bereinigte Daten (CSV): {output_csv}")


## 7. Zusammenfassung


In [ ]:
print("=" * 60)
print("ZUSAMMENFASSUNG")
print("=" * 60)
print(f"Original Datei: {file_name}")
print(f"Gesamtanzahl Zeilen (original): {len(lines)}")
print(f"Bereinigte Datenzeilen: {len(df)}")
print(f"Gelöschte Zeilen: {len(df_deleted)}")
print(f"\nGelöschte Zeilen nach Kategorie:")
for grund, count in df_deleted['Grund'].value_counts().items():
    print(f"  - {grund}: {count}")
print(f"\nOutput-Dateien:")
print(f"  - {output_csv} (CSV)")
try:
    if os.path.exists(output_excel):
        print(f"  - {output_excel} (Excel mit mehreren Sheets)")
except:
    pass
if len(df_deleted) > 0 and not os.path.exists(output_excel):
    print(f"  - {output_deleted_csv} (Gelöschte Zeilen)")
print("=" * 60)
